In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import matplotlib.pyplot as plt
from torchvision import datasets, transforms

In [2]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize(mean=0.5, std=0.5),
                              ])

trainset = datasets.FashionMNIST('F_MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = datasets.FashionMNIST('F_MNIST_data/', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

for images, labels in trainloader:
    print(type(images))
    print(images.shape)
    print(labels.shape)
    break

<class 'torch.Tensor'>
torch.Size([64, 1, 28, 28])
torch.Size([64])


In [3]:
model =  nn.Sequential(nn.Linear(784, 128),
                       nn.ReLU(),
                       nn.Linear(128, 64),
                       nn.ReLU(),
                       nn.Linear(64,10),
                       nn.LogSoftmax(dim=1))



In [4]:
criterion = nn.CrossEntropyLoss()

In [5]:
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [6]:
epochs=10

In [ ]:
for e in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        images = images.view(images.shape[0], -1)
        optimizer.zero_grad()
        output = model.forward(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    else :
        test_loss = 0
        accuracy = 0
        with torch.no_grad():
            for test_images, test_labels in testloader:
                test_images = test_images.view(test_images.shape[0], -1)
                test_output = model.forward(test_images)
                test_loss = criterion(test_output, test_labels)
                   
                proba = torch.exp(test_output)
                top_p, top_class = proba.topk(1, dim=1)
            print(f"training loss: {running_loss/len(trainloader)}")       
            print(f"test loss: {running_loss/len(testloader)}")


training loss: 0.3938910943914705
test loss: 2.353311124453499
training loss: 0.3907830980953886
test loss: 2.334742331296016
training loss: 0.37923836605603506
test loss: 2.2657680723602605
training loss: 0.3799708918340679
test loss: 2.2701445639513103
training loss: 0.3739173856657197
test loss: 2.2339777563977394
training loss: 0.3763555570293083
test loss: 2.248544665563638
training loss: 0.37031335913455055
test loss: 2.2124454195427288
training loss: 0.3638319169232713
test loss: 2.173721898560691


In [ ]:

images = images.view(images.shape[0], -1)
images,labels = next(iter(testloader))
ps = torch.exp(model(images))
print (ps.shape)